In [1]:
from torchvision import transforms as tf
from torchvision.datasets import ImageFolder

import os
import random
from PIL import Image, ImageOps
import torch

In [2]:
# 각종 LIB 충돌 방지를 위한 환경 설정

os.environ['KMP_DUPLICATE_LIB_OK']='True'

이미지 데이터 파일 준비

In [ ]:
IMG_DIR = './data/train/fear/'
RPS_DIR='./data/train/fear_arg/'

In [ ]:
TARGET_COUNT = 30000  # 목표 이미지 개수

In [8]:
# 폴더 없으면 만들기
if not os.path.exists(RPS_DIR):
    os.makedirs(RPS_DIR)

# 현재 이미지 목록 가져오기
images = os.listdir(IMG_DIR)
current_count = len(images)

# 데이터 증강 정의
augmentation = tf.Compose([
    tf.RandomRotation(degrees=(-30, 30), expand=True),   # 랜덤 회전
    tf.RandomHorizontalFlip(p=0.5),                      # 좌우 반전 (50% 확률)
    tf.RandomVerticalFlip(p=0.5),                        # 상하 반전 (50% 확률)
    tf.ToTensor()
])

def save_image(tensor_img, save_path):
    img = tf.ToPILImage()(tensor_img)
    img.save(save_path)

# 기존 이미지 저장하기
for img_name in images:
    src_path = os.path.join(IMG_DIR, img_name)
    dst_path = os.path.join(RPS_DIR, img_name)
    img = Image.open(src_path).convert('L')  # Grayscale로 변환
    img.save(dst_path)

# 데이터 증강으로 목표 개수까지 증가시키기
while current_count < TARGET_COUNT:
    img_name = random.choice(images)
    img_path = os.path.join(IMG_DIR, img_name)
    
    with Image.open(img_path) as img:
        img = img.convert('L')  # Grayscale로 변환
        tensor_img = tf.ToTensor()(img)
        
        # 증강 처리
        augmented_img = augmentation(img)
        
        # 이미지 저장
        new_img_name = f'augmented_{current_count}.png'
        save_path = os.path.join(RPS_DIR, new_img_name)
        save_image(augmented_img, save_path)
        
        current_count += 1

    if current_count % 1000 == 0:
        print(f'현재 이미지 개수: {current_count} / {TARGET_COUNT}')

print(f'\n 총 {current_count}개의 이미지가 {RPS_DIR}에 저장되었습니다.')

현재 이미지 개수: 12000 / 30000
현재 이미지 개수: 13000 / 30000
현재 이미지 개수: 14000 / 30000
현재 이미지 개수: 15000 / 30000
현재 이미지 개수: 16000 / 30000
현재 이미지 개수: 17000 / 30000
현재 이미지 개수: 18000 / 30000
현재 이미지 개수: 19000 / 30000
현재 이미지 개수: 20000 / 30000
현재 이미지 개수: 21000 / 30000
현재 이미지 개수: 22000 / 30000
현재 이미지 개수: 23000 / 30000
현재 이미지 개수: 24000 / 30000
현재 이미지 개수: 25000 / 30000
현재 이미지 개수: 26000 / 30000
현재 이미지 개수: 27000 / 30000
현재 이미지 개수: 28000 / 30000
현재 이미지 개수: 29000 / 30000
현재 이미지 개수: 30000 / 30000

 총 30000개의 이미지가 ./data/train/fear/에 저장되었습니다.


In [ ]:
import os
import random
import shutil
from PIL import Image, ImageOps
from torchvision import transforms

# 경로 설정
BASE_DIR = './data/train/'
TARGET_DIR = './data/train_balanced/'
TARGET_COUNT = 5000

# 폴더 생성하기
if not os.path.exists(TARGET_DIR):
    os.makedirs(TARGET_DIR)

# 증강 함수 정의하기
augmentation = transforms.Compose([
    transforms.RandomRotation(degrees=(-30, 30)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor()
])

def augment_image(image):
    img = image.convert('L')  # Grayscale로 변환
    tensor_img = transforms.ToTensor()(img)
    augmented_img = augmentation(img)
    return augmented_img

def process_images(emotion, image_count, augment=False):
    src_dir = os.path.join(BASE_DIR, emotion)
    dst_dir = os.path.join(TARGET_DIR, emotion)
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)

    images = os.listdir(src_dir)
    current_count = len(images)
    
    if current_count >= TARGET_COUNT:
        # 데이터가 충분하면 랜덤 샘플링하여 저장하기
        sampled_images = random.sample(images, TARGET_COUNT)
        for img_name in sampled_images:
            shutil.copy(os.path.join(src_dir, img_name), os.path.join(dst_dir, img_name))
        print(f'{emotion} - 기존 데이터에서 {TARGET_COUNT}개로 샘플링 완료.')
    
    else:
        # 데이터가 부족하면 증강하기
        for img_name in images:
            shutil.copy(os.path.join(src_dir, img_name), os.path.join(dst_dir, img_name))

        while current_count < TARGET_COUNT:
            img_name = random.choice(images)
            img_path = os.path.join(src_dir, img_name)
            with Image.open(img_path) as img:
                augmented_img = augment_image(img)
                new_img_name = f'{emotion}_augmented_{current_count}.png'
                save_path = os.path.join(dst_dir, new_img_name)
                augmented_img = transforms.ToPILImage()(augmented_img)
                augmented_img.save(save_path)
                current_count += 1
        print(f'{emotion} - 데이터 증강하여 {TARGET_COUNT}개로 맞춤 완료.')

# 감정별로 데이터 처리하기
process_images('angry', 6566)      # 줄이기
process_images('disgust', 3231, augment=True)  # 늘리기 (증강 필요)
process_images('happy', 28592)      # 줄이기
process_images('neutral', 29384)    # 줄이기
process_images('sad', 12223, augment=True)     # 늘리기 (증강 필요)
process_images('surprise', 8113)    # 줄이기


In [9]:
import os
import random
import shutil

# 경로 설정
BASE_DIR = './data/train/'

# 클래스별 목표 개수 설정
class_targets = {
    'angry': 5000,
    'disgust': 4433,  # 그대로 사용
    'fear': 30000,    # 이미 만들어져 있음
    'happy': 5567,
    'neutral': 5000,
    'sad': 5000,
    'surprise': 5000
}

def process_images(emotion, target_count):
    src_dir = os.path.join(BASE_DIR, emotion)
    images = os.listdir(src_dir)
    current_count = len(images)
    
    if current_count < target_count:
        print(f'오류: {emotion} 폴더에 이미지가 부족합니다. (현재 개수: {current_count}, 필요 개수: {target_count})')
        return

    # 남겨둘 파일을 샘플링하기 (정확히 target_count만큼)
    keep_images = random.sample(images, target_count)
    keep_images_set = set(keep_images)  # 빠르게 검색하기 위해 set으로 변환

    # 삭제할 파일 목록 생성하기
    delete_images = [img_name for img_name in images if img_name not in keep_images_set]

    # 필요 없는 이미지 삭제하기
    for img_name in delete_images:
        img_path = os.path.join(src_dir, img_name)
        os.remove(img_path)

    print(f'{emotion} 폴더가 {target_count}개로 조정되었습니다. (삭제된 이미지 개수: {len(delete_images)})')

# 클래스별로 데이터 처리하기
for emotion, target_count in class_targets.items():
    process_images(emotion, target_count)

print("\n 끝")

angry 폴더가 5000개로 조정되었습니다. (삭제된 이미지 개수: 355)
disgust 폴더가 4433개로 조정되었습니다. (삭제된 이미지 개수: 0)
fear 폴더가 30000개로 조정되었습니다. (삭제된 이미지 개수: 0)
오류: happy 폴더에 이미지가 부족합니다. (현재 개수: 5000, 필요 개수: 5567)
neutral 폴더가 5000개로 조정되었습니다. (삭제된 이미지 개수: 0)
sad 폴더가 5000개로 조정되었습니다. (삭제된 이미지 개수: 0)
surprise 폴더가 5000개로 조정되었습니다. (삭제된 이미지 개수: 1000)

 끝
